## Python Modules

In [4]:
import os
import re
import pandas as pd
import numpy as np
import spacy
import nltk
import unicodedata

from collections import Counter

from nltk.corpus import stopwords
from unidecode import unidecode

import plotly.express as px
import plotly.io as pio

# Plots renderizados en el navegador
pio.renderers.default = 'browser'

## Read data

In [5]:
archivo_csv = os.path.join('data', 'ES', 'primicias', 'noticias_primicias_32180_2024-02-01.csv')

dataset = pd.read_csv(archivo_csv)

dataset['Fecha_Publicacion'] = dataset['Fecha_Publicacion'].str.extract(r'(\d+ \w+ \d+ - \d+:\d+)', expand=False)

meses_dic = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'Apr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}

# Reemplazar el mes por su número
dataset['Fecha_Publicacion'] = dataset['Fecha_Publicacion'].str.replace(
    r' (\w+) ',
    lambda x: ' ' + meses_dic.get(x.group(1), '00') + ' ',
    regex=True
)

dataset['Fecha_Publicacion'] = pd.to_datetime(dataset['Fecha_Publicacion'], format='%d %m %Y - %H:%M', errors='coerce')

# Eliminar filas con fechas inválidas
dataset = dataset[dataset['Fecha_Publicacion'].notna()].reset_index(drop=True)

## Filtrado Fechas

In [ ]:
fecha_inicio = pd.to_datetime('2021-11-01')
fecha_fin    = pd.to_datetime('2024-01-30')

dataset = dataset[(dataset['Fecha_Publicacion'] >= fecha_inicio) & (dataset['Fecha_Publicacion'] <= fecha_fin)]

In [ ]:
fecha_minima = dataset['Fecha_Publicacion'].min()
fecha_maxima = dataset['Fecha_Publicacion'].max()

print(f"Fecha mínima: {fecha_minima}")
print(f"Fecha máxima: {fecha_maxima}")

## Drop data

In [ ]:
print(dataset.shape[0])
dataset = dataset.dropna(subset=['Fecha_Publicacion'])
print(dataset.shape[0])

In [ ]:
### https://github.com/huggingface/notebooks/blob/main/transformers_doc/en/preprocessing.ipynb
### https://huggingface.co/docs/transformers/main/en/notebooks

In [ ]:
cont_Titulo_Seccion = dataset['Titulo_Seccion'].value_counts()

fig = px.bar(
    x=cont_Titulo_Seccion.index,
    y=cont_Titulo_Seccion.values,
    labels={'x': 'Título de Sección', 'y': 'Cantidad de Noticias'},
    title='Frecuencia de Noticias por Título de Sección'
)

fig.show()

### Remove advertising data

In [ ]:
def remove_sponsored_content(texto):
    try:
        index = texto.index('Contenido Patrocinado')
        return texto[:index]
    except ValueError:
        return texto

dataset['Contenido_Parrafos'] = dataset['Contenido_Parrafos'].apply(remove_sponsored_content)

### Filtered by news category

In [ ]:
type_news = ['Economía', 'Sucesos', 'Política', 'Sociedad', 'Lo último', 'Tecnociencia', 'Internacional',
             'Seguridad', 'Elecciones presidenciales 2023', 'Seccionales 2023', 'Quito', 'Guayaquil', 'EN VIVO']
type_news = ['Economía', 'Sucesos', 'Política', 'Sociedad', 'Internacional', 'Seguridad', 'Quito', 'Guayaquil']

dataset = dataset[dataset['Titulo_Seccion'].isin(type_news)]

In [ ]:
dataset.info()

## Preprocess

In [ ]:
text = '''El Gobierno anunció que invertirá este año más de USD 516 millones en obras de infraestructura eléctrica de subtransmisión, 
distribución y alumbrado público. 35USD y 15%'''

text = unicodedata.normalize('NFD', text)
text = text.encode('ascii', 'ignore').decode('utf-8')
text = text.lower()


cleaned_text_1 = re.sub(r'[^a-z0-9\s]', '', text)
print(cleaned_text_1)

cleaned_text_2 = re.sub(r'[^\w\s]|[\d_]+', '', text)
print(cleaned_text_2)

In [ ]:
import stanza

### Descargar las stopwords y configurar el tokenizador
#nltk.download('stopwords')

### Cargar el modelo de spaCy para español
# pip install spacy
# python -m spacy download es_core_news_md

# pip install stanza
# stanza.download('es')
stop_words = set(stopwords.words('spanish'))
stop_words_unicode = {unidecode(word) for word in stop_words}

In [ ]:
# Inicializar modelos
stanza_nlp = stanza.Pipeline('es')
spacy_nlp  = spacy.load('es_core_news_md')

def preprocess_and_clean(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFD', text)
        text = text.encode('ascii', 'ignore').decode('utf-8')
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return text
    else:
        return text

def lemmatize_with_stanza(text):
    doc = stanza_nlp(text)
    return ' '.join(
        unidecode(word.lemma).lower()
        for sentence in doc.sentences
        for word in sentence.words
        if unidecode(word.lemma).lower() not in stop_words_unicode
    )

def lemmatize_with_spacy(text):
    doc = spacy_nlp(text)
    lemmatized = ' '.join([
        unidecode(token.lemma_)
        for token in doc
        if unidecode(token.lemma_) not in stop_words_unicode
    ])
    return lemmatized

def tokenize_fw(text):
    doc = spacy_nlp(text)
    return [
        token.text.lower() 
        for token in doc 
        if not token.is_space and not token.is_punct
    ]

In [ ]:
from tqdm import tqdm

tqdm.pandas(desc="Preprocesando")
dataset['data_preprocess'] = dataset['Contenido_Parrafos'].progress_apply(preprocess_and_clean)

tqdm.pandas(desc="Lematizando")
dataset['data_preprocess'] = dataset['data_preprocess'].progress_apply(lemmatize_with_spacy)
# dataset['data_preprocess'] = dataset['data_preprocess'].progress_apply(lemmatize_with_stanza)

tqdm.pandas(desc="Tokenizando")
dataset['data_tokenized'] = dataset['data_preprocess'].progress_apply(tokenize_fw)

In [ ]:
'''
dataset['data_preprocess'] = dataset['Contenido_Parrafos'].apply(preprocess_text)
dataset['data_preprocess'] = dataset['data_preprocess'].apply(lemmatize_text)
dataset['data_tokenized']  = dataset['data_preprocess'].apply(tokenize_fw)
'''

### Example of row

In [ ]:
dataset.iloc[2]

In [ ]:
### OPEN Notebook 2_Exploratory_Data_Analysis.ipynb

## Delete Head data

In [ ]:
words_to_delete = [
    'ser', 'haber', 'leer', 'tener', 'decir', 'ecuador', 'usd', 'pais', 'ano', 
    'suceso', 'segun', 'el', 'millon', '2023', 'gobierno', 'poder', 'nuevo',
    '2022', 'sociedad', 'nacional', 'dos', 'hacer', 'persona', 'dia', '2021',
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto',
    'septiembre', 'octubre', 'noviembre', 'diciembre']


def remove_words(text, words):
    pattern = r'\b(?:' + '|'.join(map(re.escape, words)) + r')\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

def remove_from_tokens(tokens, delete_list):
    return [word for word in tokens if word not in delete_list]

dataset['data_preprocess'] = dataset['data_preprocess'].apply(lambda x: remove_words(x, words_to_delete))
dataset['data_tokenized'] = dataset['data_tokenized'].apply(lambda tokens: remove_from_tokens(tokens, words_to_delete))
###https://en.wikipedia.org/wiki/Long_tail

## Save datasets

In [ ]:
archivo_csv_fw  = os.path.join('data', 'processed', 'dataset_fw.csv')

dataset_fw = dataset[['Fecha_Publicacion', 'data_tokenized']].copy()
dataset_fw = dataset_fw.dropna()
dataset_fw.to_csv(archivo_csv_fw, index=False)

In [ ]:
archivo_csv_cl = os.path.join('data', 'processed', 'dataset_cl.csv')

dataset_cl = dataset[['Titulo_Seccion', 'Fecha_Publicacion', 'Header_Tags', 'data_preprocess', 'data_tokenized']].copy()
dataset_cl = dataset_cl.dropna()
dataset_cl.to_csv(archivo_csv_cl, index=False)

## KIM & KEM

In [ ]:
# Convertir fechas a periodos diarios
dataset_cl['Fecha_Publicacion'] = pd.to_datetime(dataset_cl['Fecha_Publicacion'], format='%d %b %Y')
dataset_cl['time_segment'] = dataset_cl['Fecha_Publicacion'].dt.to_period('M')
dataset_cl['time_segment'] = dataset_cl['time_segment'].astype('category')

n = dataset_cl['time_segment'].nunique()

term_frequencies     = []
document_frequencies = []

for period, group in dataset_cl.groupby('time_segment'):
    all_words = [word for tokens in group['data_tokenized'] for word in tokens]
    term_frequency = Counter(all_words)
    document_frequency = Counter([word for tokens in group['data_tokenized'] for word in set(tokens)])
    
    for term in term_frequency:
        term_frequencies.append({
            'term': term,
            'time_segment': period,
            'term_frequency': term_frequency[term],
            'total_terms': len(all_words)
        })
        document_frequencies.append({
            'term': term,
            'time_segment': period,
            'document_frequency': document_frequency[term],
            'total_documents': len(group)
        })

tf_df = pd.DataFrame(term_frequencies)
df_df = pd.DataFrame(document_frequencies)

# Fusionar frecuencias
frequency_df = pd.merge(tf_df, df_df, on=['term', 'time_segment'])
frequency_df['time_segment'] = frequency_df['time_segment'].astype('category')

# Filtrar por umbral mínimo de frecuencia
frequency_df = frequency_df[frequency_df['term_frequency'] >= 3]

# Calcular DoV y DoD
time_weight = 0.05
frequency_df['DoV'] = (frequency_df['term_frequency'] / frequency_df['total_terms']) * (1 - time_weight * (n - frequency_df['time_segment'].cat.codes))
frequency_df['DoD'] = (frequency_df['document_frequency'] / frequency_df['total_documents']) * (1 - time_weight * (n - frequency_df['time_segment'].cat.codes))

# Agrupar por término
result_df = frequency_df.groupby('term').agg({
    'term_frequency': 'sum',
    'document_frequency': 'sum',
    'DoV': 'mean',
    'DoD': 'mean'
}).reset_index()

result_df.columns = ['Keyword', 'Total Term Frequency', 'Total Document Frequency', 'DoV', 'DoD']
result_df = result_df.sort_values(by='DoV', ascending=False)

# Calcular medias
mean_DoV = result_df['DoV'].mean()
mean_DoD = result_df['DoD'].mean()

# Crear mapas KEM y KIM
fig_kem = px.scatter(result_df, x='Total Term Frequency', y='DoV',
                     title='Keyword Emergence Map (KEM)',
                     labels={'Total Term Frequency': 'Term Frequency', 'DoV': 'Degree of Visibility (DoV)'},
                     hover_data={'Keyword': True},
                     width=800, height=600)

fig_kem.add_hline(y=mean_DoV, line_dash="dash", line_color="red")
fig_kem.add_vline(x=result_df['Total Term Frequency'].mean(), line_dash="dash", line_color="red")
fig_kem.update_traces(marker=dict(size=12, opacity=0.8))
fig_kem.update_layout(font=dict(size=15))

fig_kem.show()

fig_kim = px.scatter(result_df, x='Total Document Frequency', y='DoD',
                     title='Keyword Issue Map (KIM)',
                     labels={'Total Document Frequency': 'Document Frequency', 'DoD': 'Degree of Diffusion (DoD)'},
                     hover_data={'Keyword': True},
                     width=800, height=600)

fig_kim.add_hline(y=mean_DoD, line_dash="dash", line_color="red")
fig_kim.add_vline(x=result_df['Total Document Frequency'].mean(), line_dash="dash", line_color="red")
fig_kim.update_traces(marker=dict(size=12, opacity=0.8))
fig_kim.update_layout(font=dict(size=15))

fig_kim.show()

# Identificar señales debiles
result_df['quadrant_DoV'] = np.where((result_df['DoV'] > mean_DoV) & (result_df['Total Term Frequency'] < result_df['Total Term Frequency'].mean()), 'Weak', 'Other')
result_df['quadrant_DoD'] = np.where((result_df['DoD'] > mean_DoD) & (result_df['Total Document Frequency'] < result_df['Total Document Frequency'].mean()), 'Weak', 'Other')

# Encontrar keywords en KEM AND KIM
weak_signals_f = result_df[(result_df['quadrant_DoV'] == 'Weak') & (result_df['quadrant_DoD'] == 'Weak')]

output_file       = os.path.join('data', 'processed', 'weak_signals_kem_kim.csv')
weak_signals_f.to_csv(output_file, index=False)

print(weak_signals_f[['Keyword', 'DoV', 'DoD']])

### Keywords maps ToyExample

In [ ]:
# data = {
#     'Fecha_Publicacion': [
#         '2023-01-01 10:00', '2023-01-01 11:00', '2023-01-01 12:00',
#         '2023-01-02 10:00', '2023-01-02 11:00', '2023-01-02 12:00',
#         '2023-01-03 10:00', '2023-01-03 11:00', '2023-01-03 12:00',
#         '2023-01-04 10:00', '2023-01-04 11:00', '2023-01-04 12:00',
#         '2023-01-05 10:00', '2023-01-05 11:00', '2023-01-05 12:00',
#         '2023-01-06 10:00', '2023-01-06 11:00', '2023-01-06 12:00',
#         '2023-01-07 10:00', '2023-01-07 11:00', '2023-01-07 12:00'
#     ],
#     'Contenido_Parrafos': [
#         "Este es el primer documento. Habla sobre inteligencia artificial y aprendizaje profundo.",
#         "Este es el segundo documento. Menciona redes neuronales y procesamiento de lenguaje natural.",
#         "El tercer documento trata sobre visión por computadora y reconocimiento de imágenes.",
#         "Este cuarto documento menciona big data y análisis de datos.",
#         "El quinto documento habla sobre aprendizaje automático y algoritmos de machine learning.",
#         "Este es el sexto documento y trata sobre minería de datos y extracción de conocimiento.",
#         "El séptimo documento menciona robótica y automatización.",
#         "Este octavo documento habla sobre sistemas embebidos y IoT.",
#         "El noveno documento menciona ciberseguridad y protección de datos.",
#         "El décimo documento trata sobre blockchain y criptomonedas.",
#         "Este undécimo documento habla sobre realidad virtual y aumentada.",
#         "El duodécimo documento menciona biotecnología y genómica.",
#         "El decimotercer documento trata sobre energías renovables y sostenibilidad.",
#         "Este decimocuarto documento menciona vehículos autónomos y conducción automática.",
#         "El decimoquinto documento habla sobre inteligencia artificial y ética.",
#         "Este decimosexto documento trata sobre cloud computing y servicios en la nube.",
#         "El decimoséptimo documento menciona edge computing y procesamiento en el borde.",
#         "Este decimoctavo documento habla sobre quantum computing y computación cuántica.",
#         "El decimonoveno documento menciona software libre y código abierto.",
#         "El vigésimo documento trata sobre economía digital y comercio electrónico.",
#         "Este vigesimoprimer documento menciona marketing digital y publicidad en línea."
#     ]
# }

# datasett = pd.DataFrame(data)
# datasett['Fecha_Publicacion'] = pd.to_datetime(datasett['Fecha_Publicacion'])


# datasett['data_preprocess'] = datasett['Contenido_Parrafos'].apply(preprocess_and_clean)
# datasett['data_tokenized']  = datasett['data_preprocess'].apply(tokenize_fw)

# # Calcular frecuencias
# datasett['time_segment'] = datasett['Fecha_Publicacion'].dt.to_period('D')
# datasett['time_segment'] = datasett['time_segment'].astype('category')
# n = dataset['time_segment'].nunique()

# term_frequencies     = []
# document_frequencies = []

# for period, group in datasett.groupby('time_segment'):
#     all_words = [word for tokens in group['data_tokenized'] for word in tokens]
#     term_frequency = Counter(all_words)
#     document_frequency = Counter([word for tokens in group['data_tokenized'] for word in set(tokens)])
    
#     for term in term_frequency:
#         term_frequencies.append({'term': term, 'time_segment': period, 'term_frequency': term_frequency[term], 'total_terms': len(all_words)})
#         document_frequencies.append({'term': term, 'time_segment': period, 'document_frequency': document_frequency[term], 'total_documents': len(group)})

# tf_df = pd.DataFrame(term_frequencies)
# df_df = pd.DataFrame(document_frequencies)

# # Fusionar frecuencias
# frequency_df = pd.merge(tf_df, df_df, on=['term', 'time_segment'])
# frequency_df['time_segment'] = frequency_df['time_segment'].astype('category')

# # Calcular DoV y DoD
# time_weight = 0.05
# frequency_df['DoV'] = (frequency_df['term_frequency'] / frequency_df['total_terms']) * (1 - time_weight * (n - frequency_df['time_segment'].cat.codes))
# frequency_df['DoD'] = (frequency_df['document_frequency'] / frequency_df['total_documents']) * (1 - time_weight * (n - frequency_df['time_segment'].cat.codes))

# # Agrupar por término
# result_df = frequency_df.groupby('term').agg({
#     'term_frequency': 'sum',
#     'document_frequency': 'sum',
#     'DoV': 'mean',
#     'DoD': 'mean'
# }).reset_index()

# result_df.columns = ['Keyword', 'Total Term Frequency', 'Total Document Frequency', 'DoV', 'DoD']
# result_df = result_df.sort_values(by='DoV', ascending=False)

# # Calcular medias
# mean_DoV = result_df['DoV'].mean()
# mean_DoD = result_df['DoD'].mean()

# # Crear mapas KEM y KIM
# fig_kem = px.scatter(result_df, x='Total Term Frequency', y='DoV',
#                      title='Keyword Emergence Map (KEM)',
#                      labels={'Total Term Frequency': 'Term Frequency', 'DoV': 'Degree of Visibility (DoV)'},
#                      hover_data={'Keyword': True},
#                      width=800, height=600)

# fig_kem.add_hline(y=mean_DoV, line_dash="dash", line_color="red")
# fig_kem.add_vline(x=result_df['Total Term Frequency'].mean(), line_dash="dash", line_color="red")
# fig_kem.update_traces(marker=dict(size=12, opacity=0.8))
# fig_kem.update_layout(font=dict(size=15))

# fig_kem.show()

# fig_kim = px.scatter(result_df, x='Total Document Frequency', y='DoD',
#                      title='Keyword Issue Map (KIM)',
#                      labels={'Total Document Frequency': 'Document Frequency', 'DoD': 'Degree of Diffusion (DoD)'},
#                      hover_data={'Keyword': True},
#                      width=800, height=600)

# fig_kim.add_hline(y=mean_DoD, line_dash="dash", line_color="red")
# fig_kim.add_vline(x=result_df['Total Document Frequency'].mean(), line_dash="dash", line_color="red")
# fig_kim.update_traces(marker=dict(size=12, opacity=0.8))
# fig_kim.update_layout(font=dict(size=15))

# fig_kim.show()

# # Identificar señales debiles
# result_df['quadrant_DoV'] = np.where((result_df['DoV'] > mean_DoV) & (result_df['Total Term Frequency'] < result_df['Total Term Frequency'].mean()), 'Weak', 'Other')
# result_df['quadrant_DoD'] = np.where((result_df['DoD'] > mean_DoD) & (result_df['Total Document Frequency'] < result_df['Total Document Frequency'].mean()), 'Weak', 'Other')

# # Encontrar keywords en KEM & KIM
# weak_signals_t = result_df[(result_df['quadrant_DoV'] == 'Weak') & (result_df['quadrant_DoD'] == 'Weak')]
# print(weak_signals_t.head())